## Data dependencies

In [1]:
!sha1sum ../data/cwn-prompt-500-perm-eval.csv # 14.20
!sha1sum ../data/cwn-prompt-symtable.json # 24.05

2e101d021306406001c3c4a6c3bf3679bc4149e9  ../data/cwn-prompt-500-perm-eval.csv
3ef55f3f5ed97f86f8e3be0274e1721a01782f98  ../data/cwn-prompt-symtable.json


## Parameters

In [2]:
use_model = "twLLaMA"

In [3]:
if use_model == "cwnLLaMA":
  model_name = "cwnLlama"
  repo_name = "/mnt/md0/models/LoLLaMA/cwn-Taiwan-LLaMa/"
elif use_model == "twLLaMA":
  model_name = "twLlama"
  repo_name = "yentinglin/Taiwan-LLaMa-v1.0"
else:
  raise ValueError("Invalid model name")
print(model_name)

twLlama


## Load Data

In [4]:
import os
import sys
os.environ["BNB_CUDA_VERSION"] = "115"
if "../src/llm_compressor/src" not in sys.path:
  sys.path.append("../src/llm_compressor/src")

In [5]:
from pathlib import Path
import json
import pickle
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, GPTNeoXForCausalLM, AutoTokenizer
from huggingface_hub import list_repo_refs 

from llm_compressor import AECompressorLLM

/home/seantyh/miniconda3/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:106: UserWarning: 

BNB_CUDA_VERSION=XXX can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64
Loading CUDA version: BNB_CUDA_VERSION=115


  warn((f'\n\n{"="*80}\n'


In [6]:
data = pd.read_csv("../data/cwn-prompt-500-perm-eval.csv")
symtable = json.loads(Path("../data/cwn-prompt-symtable.json").read_text())
rev_symtable = {v: int(k) for k, v in symtable.items()}
symlist = list(symtable.values())
n_symbol = len(symlist)
symbol_bits = np.log2(n_symbol)

## Inference

### Setup

In [7]:
# reformat the rev_list in reverse order
if use_model == "twLLaMA":
  cache_dir = Path("/home/seantyh/hdd/hf_cache/twLLaMA/main")
else:
  cache_dir = None

In [8]:
tokenizer = AutoTokenizer.from_pretrained(repo_name)

model = AutoModelForCausalLM.from_pretrained(
  repo_name,
  cache_dir=cache_dir,
  load_in_8bit=True
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/seantyh/miniconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/seantyh/miniconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [9]:
from collections import namedtuple
VarLoc = namedtuple("VarLoc", ("name", "value", "start", "end"))

def make_locmap(inst):
  return {
    x[0]: VarLoc(*x)
    for x in inst["var_loc"]
  }

def find_token_locs(batch, char_locs):
  locs = [
    batch.char_to_token(0, x)
    for x in char_locs
  ]
  if locs[-1] is None:
    locs[-1] = len(batch.input_ids[0])-1
  return locs

### Decoding

In [ ]:
proc_data = []
for _, row in tqdm(data.iterrows(), total=len(data)):
    prompt = row.prompt
    batch = tokenizer(prompt, 
                    truncation=True,
                    max_length=1000,
                    return_tensors="pt").to("cuda")
    with torch.no_grad():
        out = model(**batch, output_hidden_states=True)


    b_start, b_end = [int(x) for x in row.resp_loc[1:-1].split(",")]
    b_toklocs = find_token_locs(batch, (b_start, b_end))

    last_hidden = out.hidden_states[-1]
    vec_b = last_hidden[0, b_toklocs[0]:b_toklocs[1]].mean(0)
    logits = out.logits
    data_ids = batch.input_ids.squeeze().to("cpu").tolist()

    ab_logits = logits[:, :, torch.tensor(symlist)]
    ab_data_ids = [rev_symtable[x] for x in data_ids]
    ab_input_ids = torch.tensor(ab_data_ids).view(1, -1).to("cuda")

    probs = torch.softmax(ab_logits, dim=1).squeeze().to("cpu")
    uniform_prob = (torch.ones(probs.shape[1]) / probs.shape[1]).to("cpu")
    next_token_probs = torch.concat([uniform_prob.unsqueeze(0), probs[:-1, :]], dim=0).cpu()     
    tok_logits = ab_logits.gather(2, ab_input_ids[:, 1:].unsqueeze(2)).squeeze().to("cpu")
    tok_logits = torch.cat([torch.zeros(1,), tok_logits], dim=0)
    logit_b = tok_logits[b_toklocs[0]:b_toklocs[1]].mean(0).item()
    tok_probs = next_token_probs.gather(1, ab_input_ids.transpose(1,0).cpu()).squeeze().to("cpu")
    prob_b = tok_probs[b_toklocs[0]:b_toklocs[1]].mean(0).item()    

    # do compression
    compressor = AECompressorLLM()

    # only compress the different part of emp/perm
    ab_data_ids = ab_data_ids[b_toklocs[0]:]
    next_token_probs = next_token_probs[b_toklocs[0]:]
    try:
        msg = compressor.compress(ab_data_ids, next_token_probs)
        recon = compressor.decompress(msg, len(ab_data_ids), next_token_probs)
        compress_bits = len(msg)
        compress_ratio = compress_bits / (len(ab_data_ids) * symbol_bits)

        assert all(a==b for a, b in zip(recon, ab_data_ids))
    except Exception as ex:
        compress_bits = float('nan')
        compress_ratio = float('nan')
        print(type(ex).__name__, str(ex))        
        
    # put vectors, logits, input_ids into inst_x
    inst_x = {**row.to_dict(), 
        "logit_b": logit_b,
        "prob_b": prob_b,
        "tok_logits": tok_logits.numpy(),
        "tok_probs": tok_probs.numpy(),        
        "compress_bits": compress_bits,
        "compress_ratio": compress_ratio,
        "input_ids": batch.input_ids.cpu().numpy(),
        "ab_data_ids": ab_data_ids,
        "n_ab_data_ids": len(ab_data_ids),
    }
    proc_data.append(inst_x)
    

In [ ]:
emp_cr = np.array([x["compress_ratio"] for x in proc_data if x["prompt_type"] == "emp"])
perm_cr = np.array([x["compress_ratio"] for x in proc_data if x["prompt_type"] == "perm"])
emp_cr.mean(), perm_cr.mean()

(0.715602938969356, 0.7959351808503101)

## Export artefacts

In [ ]:
out_dir = Path("../data/minrep/")
out_dir.mkdir(exist_ok=True, parents=True)
out_path = out_dir / f"minrep-{model_name}-cwn-eval.pkl"
fsize = out_path.write_bytes(pickle.dumps(proc_data))
print("artefact size (MB):", fsize//(1024**2))

artefact size (MB): 49


In [ ]:
!sha1sum {str(out_path)}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
38892c258a249961395c0a0a7682683c65a89f99  ../data/minrep/minrep-pythia-6.9b.pkl
